In [77]:
import plotly.io as pio
import plotly.express as px
import pandas as pd
import json
import suntime
import regex 
import maidenhead
from datetime import datetime
import time

In [78]:
pio.renderers.default='browser'

In [79]:
#VARIABLES

testGridSquare = "CN96rp"
testSquare = maidenhead.to_location(testGridSquare, center=True)

#maidenhead returns 
gridWidthInLong = 2;
gridHeightInLat = 1;


In [80]:
#dataframe to hold log data
logDF = pd.DataFrame(columns=['Callsign', 'GridSquare', 'GridSubSquare', 'Time'])


#read in WSJT's All.txt decoded message log
with open("C:/Users/cutup/AppData/Local/WSJT-X/ALL.txt") as FT8Log:
    lines = FT8Log.readlines()
    
#regex based callsign and gridsquare extraction
for line in lines[:]:
    
    match = ""
    gridSq = ""
    
    #callsign matches (international)
    matches = regex.findall("[a-zA-Z0-9]{1,3}[0-9][a-zA-Z0-9]{0,3}[a-zA-Z]", line)
    
    #if two call signs are heard we take the latter as this is the message sender replying
    if len(matches) > 1:
        match = matches[1]
    elif len(matches) == 1:
        match = matches[0]
    
    #gridsquare match
    if(match != ""):
        gridSqMatch = regex.search("[A-R]{2}[0-9]{2}", line)

        #if found gets the match
        if(gridSqMatch):
            gridSq = gridSqMatch.group()

        #currently only getting rows with reported gridsquare, will add other categories
        #RR73 just happens to be a signoff message and a grid square, it is way out of the way so
        #ignoring for now
        if (gridSq != "" and gridSq != "RR73"):
            tokens = line.split()
            timeString = tokens[0]
            dateTime = datetime.strptime(timeString, '%y%m%d_%H%M%S')
            logDF = logDF.append({'Callsign':match, 'GridSquare':gridSq, 'GridSubSquare':"",
                                  'Time': dateTime, 'Month': dateTime.month, 'Day':dateTime.day, 'Hour':dateTime.hour,
                                  'Db': int(tokens[4])}, ignore_index=True)
        

logDF.head() #displays


,Callsign,GridSquare,GridSubSquare,Time,Day,Db,Hour,Month
0,AD5CQ,EL29,,2023-04-07 18:26:30,7.0,18.0,18.0,4.0
1,WA6EZV,EM79,,2023-04-07 18:26:45,7.0,3.0,18.0,4.0
2,WA6EZV,EM79,,2023-04-07 18:27:15,7.0,-4.0,18.0,4.0
3,KG4LKY,EM87,,2023-04-07 18:27:15,7.0,-7.0,18.0,4.0
4,AD5CQ,EL29,,2023-04-07 18:28:15,7.0,9.0,18.0,4.0


In [81]:
logDF.dtypes


Callsign                 object
GridSquare               object
GridSubSquare            object
Time             datetime64[ns]
Day                     float64
Db                      float64
Hour                    float64
Month                   float64
dtype: object

In [82]:

gridHitsDF = logDF.groupby(logDF.GridSquare)\
            .agg({'Callsign':'nunique', 'Db':'mean'})\
            .rename(columns={"Callsign": "Callsigns", "Db": "DbMean"})
                  
gridHitsDF = gridHitsDF.reset_index()

gridHitsDF[0:10]

,GridSquare,Callsigns,DbMean
0,AD13,1,-9.000000
1,AF50,1,0.000000
2,BG08,1,-2.500000
3,BK29,1,-14.714286
4,BL01,1,-13.000000
5,BL11,2,-15.000000
6,BN35,1,-11.000000
7,BO49,1,-2.200000
8,BP51,5,-9.446809
9,BP64,1,-6.428571


In [83]:
def addLatLonFromMaidenhead(df):
    latLon = maidenhead.to_location(df["GridSquare"])
    df["lat"] = latLon[0]
    df["lon"] = latLon[1]
    return df

gridHitsDF = gridHitsDF.apply(addLatLonFromMaidenhead, axis=1)

gridHitsDF.head()


,GridSquare,Callsigns,DbMean,lat,lon
0,AD13,1,-9.000000,-57.0,-178.0
1,AF50,1,0.000000,-40.0,-170.0
2,BG08,1,-2.500000,-22.0,-160.0
3,BK29,1,-14.714286,19.0,-156.0
4,BL01,1,-13.000000,21.0,-160.0


In [84]:
def AddSunriseSunsetData(gridSquareDataFrame):
    
    #next row
    try:
        locationSunData = suntime.Sun(gridSquareDataFrame["lat"], gridSquareDataFrame["lon"])
        gridSquareDataFrame["sunrise"] = locationSunData.get_local_sunrise_time()
        gridSquareDataFrame["sunset"] = locationSunData.get_local_sunset_time()
    except:
        gridSquareDataFrame["sunrise"] = "NA"
        gridSquareDataFrame["sunset"] = "NA"
    
    return gridSquareDataFrame


gridHitsDF = gridHitsDF.apply(AddSunriseSunsetData, axis=1)


In [85]:
gridHitsDF.head()

,GridSquare,Callsigns,DbMean,lat,lon,sunrise,sunset
0,AD13,1,-9.000000,-57.0,-178.0,2023-04-17 11:52:00-07:00,2023-04-16 21:50:00-07:00
1,AF50,1,0.000000,-40.0,-170.0,2023-04-17 10:51:00-07:00,2023-04-16 21:47:00-07:00
2,BG08,1,-2.500000,-22.0,-160.0,2023-04-17 09:53:00-07:00,2023-04-16 21:26:00-07:00
3,BK29,1,-14.714286,19.0,-156.0,2023-04-17 09:06:00-07:00,2023-04-16 21:42:00-07:00
4,BL01,1,-13.000000,21.0,-160.0,2023-04-17 09:20:00-07:00,2023-04-16 22:00:00-07:00


In [86]:
#merge back in
logDF = logDF.merge(gridHitsDF[["GridSquare", "sunrise", "sunset", "lat", "lon"]], how="left", on="GridSquare")

In [87]:
logDF.head()

,Callsign,GridSquare,GridSubSquare,Time,Day,Db,Hour,Month,sunrise,sunset,lat,lon
0,AD5CQ,EL29,,2023-04-07 18:26:30,7.0,18.0,18.0,4.0,2023-04-17 04:56:00-07:00,2023-04-16 17:51:00-07:00,29.0,-96.0
1,WA6EZV,EM79,,2023-04-07 18:26:45,7.0,3.0,18.0,4.0,2023-04-17 04:05:00-07:00,2023-04-16 17:23:00-07:00,39.0,-86.0
2,WA6EZV,EM79,,2023-04-07 18:27:15,7.0,-4.0,18.0,4.0,2023-04-17 04:05:00-07:00,2023-04-16 17:23:00-07:00,39.0,-86.0
3,KG4LKY,EM87,,2023-04-07 18:27:15,7.0,-7.0,18.0,4.0,2023-04-17 04:00:00-07:00,2023-04-16 17:12:00-07:00,37.0,-84.0
4,AD5CQ,EL29,,2023-04-07 18:28:15,7.0,9.0,18.0,4.0,2023-04-17 04:56:00-07:00,2023-04-16 17:51:00-07:00,29.0,-96.0


In [96]:
test = list()

def buildGridSquareGeoJSON(df):
    test.append(df["GridSquare"])
    
gridHitsDF.apply(buildGridSquareGeoJSON, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
597    None
598    None
599    None
600    None
601    None
Length: 602, dtype: object

In [97]:
test

['AD13',
 'AF50',
 'BG08',
 'BK29',
 'BL01',
 'BL11',
 'BN35',
 'BO49',
 'BP51',
 'BP64',
 'BP74',
 'CF23',
 'CH01',
 'CH42',
 'CJ65',
 'CM87',
 'CM88',
 'CM95',
 'CM96',
 'CM97',
 'CM98',
 'CM99',
 'CN80',
 'CN82',
 'CN83',
 'CN84',
 'CN85',
 'CN86',
 'CN87',
 'CN88',
 'CN89',
 'CN94',
 'CN96',
 'CN97',
 'CN98',
 'CR61',
 'DL10',
 'DL64',
 'DL74',
 'DL80',
 'DL82',
 'DL90',
 'DL92',
 'DL95',
 'DL96',
 'DM03',
 'DM04',
 'DM05',
 'DM06',
 'DM09',
 'DM11',
 'DM12',
 'DM13',
 'DM14',
 'DM15',
 'DM16',
 'DM22',
 'DM24',
 'DM25',
 'DM26',
 'DM33',
 'DM34',
 'DM35',
 'DM37',
 'DM38',
 'DM41',
 'DM42',
 'DM43',
 'DM44',
 'DM52',
 'DM56',
 'DM57',
 'DM58',
 'DM59',
 'DM61',
 'DM62',
 'DM64',
 'DM65',
 'DM66',
 'DM67',
 'DM69',
 'DM72',
 'DM75',
 'DM78',
 'DM79',
 'DM81',
 'DM89',
 'DM91',
 'DM94',
 'DM95',
 'DM98',
 'DN06',
 'DN13',
 'DN17',
 'DN18',
 'DN27',
 'DN30',
 'DN40',
 'DN41',
 'DN43',
 'DN45',
 'DN49',
 'DN55',
 'DN60',
 'DN61',
 'DN70',
 'DN71',
 'DN80',
 'DN83',
 'DN85',
 'DN94',
 

In [103]:
#VISUALIZATION OF DECODED GRIDSQUARES
def get_polygon(lon, lat, id):
    
    coords = [[lon-gridWidthInLong/2, lat-gridHeightInLat/2], [lon+gridWidthInLong/2, lat-gridHeightInLat/2], [lon+gridWidthInLong/2, lat+gridHeightInLat/2], [lon-gridWidthInLong/2, lat+gridHeightInLat/2]]
    
    return { "type": "Feature",
                               "geometry": {"type": "Polygon",
                                            "coordinates": [coords]}, "id":id}


geojd = {"type": "FeatureCollection"}
geojd['features'] = []


#grid station map geojson
def buildGridSquareGeoJSON(df):
    geojd['features'].append(get_polygon(lon=df["lon"], lat=df["lat"], id=df["GridSquare"]))
    
gridHitsDF.apply(buildGridSquareGeoJSON, axis=1)
    


fig = px.choropleth_mapbox(gridHitsDF,geojson=geojd,locations="GridSquare",color="DbMean",
                           color_continuous_scale="Viridis",
                           range_color=(-25, 25),
                           mapbox_style="carto-positron",
                           zoom=1, center = {"lat": 80, "lon": -77},
                           opacity=0.5,
                           labels={'ID':'Grid Square'}
                          )


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [100]:
geojd

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-179.0, -57.5],
      [-177.0, -57.5],
      [-177.0, -56.5],
      [-179.0, -56.5]]]},
   'id': 'RR01'},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-171.0, -40.5],
      [-169.0, -40.5],
      [-169.0, -39.5],
      [-171.0, -39.5]]]},
   'id': 'RR01'},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-161.0, -22.5],
      [-159.0, -22.5],
      [-159.0, -21.5],
      [-161.0, -21.5]]]},
   'id': 'RR01'},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-157.0, 18.5],
      [-155.0, 18.5],
      [-155.0, 19.5],
      [-157.0, 19.5]]]},
   'id': 'RR01'},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-161.0, 20.5],
      [-159.0, 20.5],
      [-159.0, 21.5],
      [-161.0, 21.5]]]},
   'id': 'RR01'},
  {'type': 'Feature',
   'geometry': {'ty